In [2]:
#required imports
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

In [3]:
#open data file and read data
with open("ASTR19_S22_group_project_data.txt", encoding="utf-8") as file:
    raw_data = file.read()

#close file when done
file.close()

In [4]:
#init array to store each line
lineArray = []
#init place for the popped values to go
junk = ""

#split raw data into lines, store in prepared array
lineArray = str.split(raw_data, "\n")

#remove empty strings from data
for i in lineArray:
    if i == "":
        lineArray.remove(i)

#remove header information in first three lines
junk = lineArray.pop(0)
junk = lineArray.pop(0)
junk = lineArray.pop(0)

In [8]:
#init arrays to hold components of the data
dayArray = []
timeArray = []
rawDataArray = []
hold = []

#split data components and place in arrays
for i in lineArray:
    hold = str.split(i, " ")
    #this split method is not ideal, but it's been ages and all the stuff I remember is in Java.
    #as such, this loop checks for blank strings and removes them.
    for j in hold:
        if j == '':
            hold.remove('')
    #still in the for loop, add each separated data component to the discrete arrays 
    dayArray.append(hold[0])
    timeArray.append(hold[1])
    rawDataArray.append(hold[2])

In [10]:
#init array to hold time as one value, with minutes as units
minuteArray = []
#init array to hold data as float rather than str
dataArray = []
#init a holder and counter
temp = 0

#extract hours and minutes separately, convert to minutes and sum
for i in timeArray:
    hold = str.split(i, ":")
    temp = int(hold[1]) + (60 * int(hold[0]))
    minuteArray.append(temp)

#reset for use as counter
temp = 0

for i in timeArray:
    hold = str.split(i, ":")
    decimalTemp = (float(hold[0]) / 24) + ((float(hold[1]) / 24) / 60)
    decimalDayArray[temp] = decimalDayArray[temp] + decimalTemp
    temp = temp + 1

#cast rawDataArray strings to float into new array
for i in rawDataArray:
    dataArray.append(float(i))

In [ ]:
print(minuteArray)
print(dataArray)
print(dayArray)

[1927, 2377, 3415, 3865, 4905, 5354, 6395, 6840, 7888, 8326, 9382, 9811, 10884, 11296, 11722, 12781, 13207, 14266, 14690, 15118, 16171, 16616, 17648, 18105, 19127, 19585, 20602, 21060, 22079, 22534, 23554, 24008, 25030, 25481, 26505, 26952, 27981, 28425, 29457, 29895, 30938, 31368, 32425, 32840, 33262, 34312, 34738, 35790, 36217, 36644, 37703, 38155, 39195, 39657, 40689, 41150, 42185, 42639, 43678, 44127, 45170, 45610, 46660, 47092, 48150, 48574, 49639, 50052, 51130, 51529, 51932, 53004, 53413, 54480, 54894, 55314, 56379, 56835, 57869, 58342, 59361, 59834]
[6.57, -0.39, 5.38, -1.74, 5.26, -1.36, 4.13, -1.15, 3.48, -1.61, 2.62, 0.05, 1.83, 1.01, 3.57, 0.73, 4.54, 2.17, 3.76, 2.88, 3.9, 1.92, 4.73, 1.74, 5.66, 0.79, 5.33, -0.04, 5.23, -1.12, 6.67, -0.61, 4.74, -0.93, 4.69, -1.29, 4.2, 0.22, 3.68, 0.26, 2.47, 0.36, 2.59, 0.99, 4.1, 1.87, 3.17, 1.6, 3.71, 3.1, 4.04, 2.49, 4.07, 2.18, 5.93, 0.7, 6.89, 0.21, 6.72, 0.22, 6.77, -1.68, 5.77, -1.32, 4.35, -0.73, 4.23, -0.38, 2.24, -0.61, 3.45, -

In [19]:
ax.errorbar(decimalDayArray, dataArray, yerr=.25, fmt='o')
ax.set_xlabel('Time(days)')
ax.set_ylabel('Tide Height(ft)')

#finding line of best fit
#a = 3.4
#b = 2.1
#c = 2
sig = 0.6
y_err = np.full(85,sig)

def f_line(x, a, b, c):
    return a * np.sin(b*x + c)

params, params_cov = optimize.curve_fit(f_line, decimalDayArray, dataArray)

a_fit = params[0]
b_fit = params[1]
c_fit = params[2]
d_fit = params[3]

#print(a_fit,b_fit,c_fit)
#y_fit = a_fit * np.sin(b_fit * decimalDayArray + c_fit)

y_fit = []
temp = 0

for i in decimalDayArray:
    y_fit.append(a_fit * np.sin(b_fit * i) + c_fit)
    
ax.plot(decimalDayArray, y_fit)